Dataset from: https://www.kaggle.com/datasets/biancaferreira/african-wildlife
The objective was image classification between different animals, given that 
there were four animals in the dataset, therefore, four classes. When using
two different animals and the zebra dataset is involved, it presents high accuracy,
when more animals are involved, the accuracy starts to diminish, here are the accuracy
results for the different combinations possible within the dataset

buffalo x zebra : 0.95
buffalo x elephant: 0.75
buffalo x rhino: 0.48

elephant x zebra: 0.94
elephant x buffalo: 0.75
elephant x rhino: 0.49

rhino x zebra: 0.96
rhino x buffalo: 0.48
rhino x elephant: 0.49

rhino x zebra x elephant: 0.75
buffalo x zebra x elephant: 0.81
buffalo x rhino x elephant: 0.67

buffalo x zebra x elephant x rhino: 0.71

In [ ]:
import pandas as pd
import numpy as np
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout
from keras.models import Sequential
import random

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [ ]:
classes = ['buffalo', 'elephant', 'rhino', 'zebra']

In [ ]:
# images = [[imread(os.path.join('./'+direc+'/',image)) for image in os.listdir('./'+direc+'/') if image.endswith('.jpg')] for direc in classes]

In [ ]:
images = []
labels = []
for direct in classes:
    for filename in os.listdir('./'+direct+'/'):
        if filename.endswith('.jpg'):
            image = Image.open(os.path.join('./'+direct+'/', filename))
            image = image.resize((128, 128), Image.ANTIALIAS)
            image = np.asarray(image)
            # im1 = image
            # im2 = tf.image.flip_up_down(image)
            # im3 = tf.image.rot90(im1, 1)
            # im4 = tf.image.flip_left_right(image)
            images.append(image)
            label = classes.index(direct)
            labels.append(label)

In [ ]:
X = np.asarray(images)
y = np.asarray(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
X_train, X_vali, y_train, y_vali = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
tmp_x = []
tmp_y = []
for i in range(X_train.shape[0]):
    im0 = (X_train[i])
    im1 = (tf.image.flip_up_down(X_train[i]))
    im2 = (tf.image.rot90(X_train[i], 1))
    im3 = (tf.image.flip_left_right(X_train[i]))
    im4 = (keras.preprocessing.image.random_zoom(X_train[i], (.5,.4)))
    im5 = (keras.preprocessing.image.random_shear(im3,25))
    label = y_train[i]
    tmp_x.extend([im0, im1, im2, im3, im4, im5])
    tmp_y.extend([label, label, label, label, label, label])
    

In [ ]:
idx = list(range(len(tmp_x)))
random.shuffle(idx)

In [ ]:
X_train = []
y_train = []
for i in idx:
    X_train.append(tmp_x[i])
    y_train.append(tmp_y[i])

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [ ]:
X_train.shape

In [ ]:
i=1218
example = X_train[i]
# example = np.rot90(example, 1)
imshow(example)
print(classes[y_train[i]])

In [ ]:
X_train, X_test, X_vali = X_train / 255.0, X_test / 255.0, X_vali / 255.0

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D())


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(classes)))

In [ ]:
X_train.shape

In [ ]:

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_vali, y_vali))

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(test_acc)